# Projekt 1.

## Przygotowanie danych.

Domyślnie wczytuję dane ze zmiennymi które arbitralnie sobie wybrałem - zmienne kategoryczne przekształcone do macierzy zer i jedynek (one-hot encoding). Czyli nie ma zmiennej "linia lotnicza", ale jest zmienna "Is_Ryanair", "Is_LOT" - itp., zmienne binarne. Oczywiście zwykłe numeryczne też są. 

In [12]:
import pandas as pd
import numpy as np

# ostateczne dane do przewidywania typu linii lotniczej - low cost czy tradycyjna
dane = pd.read_pickle("dane_klasyfikacja_onehot.pkl")

dane.head()

,Departure_time,Arrival_time,Flight_time,Price,Num_Layovers,Cabin_bag,Checked_bag,Days_to_departure,layover_duration,Is_Departure_Warszawa,...,Is_Flight_Thursday,Is_Flight_Tuesday,Is_Flight_Wednesday,Is_Extraction_Monday,Is_Extraction_Saturday,Is_Extraction_Sunday,Is_Extraction_Thursday,Is_Extraction_Tuesday,Is_Extraction_Wednesday,Is_low-cost
0,16.58,17.92,1.33,1643,0,1,0,2,0.0,True,...,False,False,False,False,True,False,False,False,False,False
2,19.67,21.00,1.33,1749,0,1,0,2,0.0,True,...,False,False,False,False,True,False,False,False,False,False
8,9.42,10.75,1.33,1031,0,1,0,3,0.0,True,...,False,True,False,False,True,False,False,False,False,False
9,19.67,21.00,1.33,1009,0,1,0,3,0.0,True,...,False,True,False,False,True,False,False,False,False,False
10,17.67,19.00,1.33,1009,0,1,0,3,0.0,True,...,False,True,False,False,True,False,False,False,False,False


In [13]:
dane.shape
dane.columns

Index(['Departure_time', 'Arrival_time', 'Flight_time', 'Price',
       'Num_Layovers', 'Cabin_bag', 'Checked_bag', 'Days_to_departure',
       'layover_duration', 'Is_Departure_Warszawa', 'Departure_time',
       'Arrival_time', 'Flight_time', 'Price', 'Num_Layovers', 'Cabin_bag',
       'Checked_bag', 'Days_to_departure', 'layover_duration',
       'Is_Arrival_Londyn', 'Is_Arrival_Paryż', 'Is_Arrival_Rzym',
       'Is_Flight_Monday', 'Is_Flight_Saturday', 'Is_Flight_Sunday',
       'Is_Flight_Thursday', 'Is_Flight_Tuesday', 'Is_Flight_Wednesday',
       'Is_Extraction_Monday', 'Is_Extraction_Saturday',
       'Is_Extraction_Sunday', 'Is_Extraction_Thursday',
       'Is_Extraction_Tuesday', 'Is_Extraction_Wednesday', 'Is_low-cost'],
      dtype='object')

In [14]:
from sklearn.model_selection import train_test_split

# target variable - cena lotu (zł)
X = dane.drop(columns =["Is_low-cost"])
y = dane["Is_low-cost"]

In [15]:
# podział na zbiór treningowy, walidacyjny i testowy - proporcje 80% : 20% : 20% 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.75)


# konwersja do macierzy numpy
X_train_np = X_train.to_numpy().astype(np.float64)
y_train_np = y_train.to_numpy().astype(np.float64)

X_val_np = X_val.to_numpy().astype(np.float64)
y_val_np = y_val.to_numpy().astype(np.float64)
X_test_np = X_test.to_numpy().astype(np.float64)
y_test_np = y_test.to_numpy().astype(np.float64)


In [16]:
y_val_np.shape

(4846,)

## Testowanie parametrów

Grid Search - przeszukiwanie kombinacji parametrów. 

In [17]:
import itertools
from glob import glob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
import numpy as np
from perceptron_nn_classifier import MultiLayerPerceptronClassifier

num_repetitions = 5
num_classes = len(np.unique(y_test_np))

baseline_params = {
    "num_inputs": [X_test_np.shape[1]], # tego nie ruszać
    "num_layers": [[i] for i in range(1,3)], # od 1 do 20 warstw
    "learning_rate": [ 0.1],
    "num_epochs": [2000],
    "activation_function": ["relu"]
}

keys = list(baseline_params.keys())
combinations = list(itertools.product(*(baseline_params[key] for key in keys)))

# ramka danych z parametrami
params_df = pd.DataFrame(data=combinations, columns=keys)

# pusta lista na wyniki
results = []
for i, row in params_df.iterrows():
    try:
        num_inputs = row["num_inputs"]
        num_layers = row["num_layers"]
        learning_rate = row["learning_rate"]
        activation_function = row["activation_function"]
        num_epochs = row["num_epochs"]
        
        # żeby sprawdzić wszystkie parametry metody fit, idź do perceptron_nn.ipynb
        epochs = []
        train_loss = []
        train_accuracy = []
        val_loss = []
        val_accuracy = []
        test_accuracy = []
        test_precision = []
        test_recall = []
        test_f1 = []
        test_log_loss = []
        
        for k in range(num_repetitions):
            print(f"{k+1} powtórzenie dla kombinacji {i+1}/{len(params_df)}")
            mlp = MultiLayerPerceptronClassifier(
                num_inputs=num_inputs,
                num_classes=num_classes, 
                num_layers=num_layers, 
                learning_rate=learning_rate,
                activation_function=activation_function)
            
            mlp.fit(
                X=X_train_np,
                y=y_train_np,  # już w formacie one-hot
                X_val=X_val_np,
                y_val=y_val_np,  # już w formacie one-hot
                num_epochs=num_epochs,
                verbose=True) 
            
            res = mlp.history
            
            # Przewidywania
            pred_classes = mlp.predict(X_test_np)  # klasy (0, 1, 2, ...)
            pred_proba = mlp.predict_proba(X_test_np)  # prawdopodobieństwa
            
            # Konwersja one-hot na klasy dla prawdziwych etykiet
            y_test_classes = np.argmax(y_test_np, axis=1)
            
            # Zapisanie wyników
            epochs.append(res["epoch"])
            train_loss.append(res["train_loss"])
            train_accuracy.append(res["train_accuracy"])
            val_loss.append(res["val_loss"])
            val_accuracy.append(res["val_accuracy"])
            
            # Metryki testowe
            test_accuracy.append(accuracy_score(y_test_classes, pred_classes))
            test_precision.append(precision_score(y_test_classes, pred_classes, average='weighted', zero_division=0))
            test_recall.append(recall_score(y_test_classes, pred_classes, average='weighted', zero_division=0))
            test_f1.append(f1_score(y_test_classes, pred_classes, average='weighted', zero_division=0))
            
            # Log loss (cross-entropy)
            try:
                test_log_loss.append(log_loss(y_test_np, pred_proba))
            except:
                test_log_loss.append(np.nan)  # w przypadku problemów numerycznych
        
        results.append({
            "num_inputs": num_inputs,
            "num_classes": num_classes,
            "num_layers": num_layers,
            "learning_rate": learning_rate,
            "num_epochs": epochs,  # średnia liczba epok (może się różnić przez early stopping)
            "activation_function": activation_function,
            "train_loss": np.mean(train_loss),
            "train_accuracy": np.mean(train_accuracy),
            "val_loss": np.mean(val_loss),
            "val_accuracy": np.mean(val_accuracy),
            "test_accuracy": np.mean(test_accuracy),
            "test_precision": np.mean(test_precision),
            "test_recall": np.mean(test_recall),
            "test_f1": np.mean(test_f1),
            "test_log_loss": np.mean(test_log_loss)
        })
        
        print(f"Ukończono kombinację {i+1}/{len(params_df)}: "
              f"Test Accuracy = {np.mean(test_accuracy):.4f} ± {np.std(test_accuracy):.4f}")
              
    except KeyboardInterrupt:
        print("Przerwano przez użytkownika")
        break

result_df = pd.DataFrame(results)

files = glob("nn_klasyfikacja_*.xlsx")
# zapis wyników do pliku
filename = f"nn_klasyfikacja_{len(files)+1}.xlsx"
result_df.to_excel(filename, index=False)
print(f"Wyniki zapisane do pliku: {filename}")

1 powtórzenie dla kombinacji 1/2
Architektura sieci: Wejścia(34) -> Warstwa_1(1) -> Wyjście(2)


ValueError: operands could not be broadcast together with shapes (14536,) (14536,2) 